In [366]:
from datetime import datetime

floorsheet_url  = "http://www.nepalstock.com/main/floorsheet/index/page/?contract-no=&stock-symbol=&buyer=&seller=&_limit=20000"
floorsheet_columns = ["Contact No", "Stock Symbol", "Buyer Broker", "Seller Broker", "Quantity", "Rate", "Amount"] 
df = pd.DataFrame(columns = floorsheet_columns)
pages = 3

for i in range(pages):
    url = floorsheet_url.replace("page", str(i+1))
    data = pd.read_html(url)[0]
    data = data.iloc[2:-3, 1:-2]    
    data.columns = floorsheet_columns
    df = df.append(data)    
df.set_index("Contact No")

data = df.copy()
data = data.apply(lambda row: pd.Series([int(row["Contact No"]), row["Stock Symbol"], 
                        row["Buyer Broker"], row["Seller Broker"],
                        int(row["Quantity"]), float(row["Rate"]),
                        float(row["Amount"])
                       ]), axis = 1)
data.columns = floorsheet_columns
data.set_index("Contact No")

pd.set_option('display.float_format', lambda x: '%.1f' % x)
trade_data  = data.copy()
trade_data.pop("Contact No")
trade_data.pop("Rate")

buy_stat = trade_data.groupby(["Buyer Broker", "Stock Symbol"]).sum()
sell_stat = trade_data.groupby(["Seller Broker", "Stock Symbol"]).sum()

buy_stat.index.names = ["Broker", "Stock Symbol"]
sell_stat.index.names = ["Broker", "Stock Symbol"]
trade_stat = buy_stat.sub(sell_stat, fill_value = 0)
trade_stat["Average Rate"] = trade_stat["Amount"]/trade_stat["Quantity"]
trade_stat.to_csv(datetime.today().strftime('%Y-%m-%d') + ".csv")

In [365]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2021-11-16'

,Stock Symbol,Buyer Broker,Seller Broker,Quantity,Rate,Amount
Contact No,,,,,,
2021111503026456,NLIC,38,6,20,1608.00,32160.00
2021111503026455,CHDC,32,38,20,868.80,17376.00
2021111503026454,HIDCL,58,32,10,329.50,3295.00
2021111503026453,HIDCL,45,32,40,329.50,13180.00
2021111503026452,NIFRA,4,37,60,355.80,21348.00
...,...,...,...,...,...,...
2021111501000001,SCB,53,22,21,507.80,10663.80
2021111503000002,CBBL,22,47,30,1754.00,52620.00
2021111505000001,ULI,1,7,10,620.00,6200.00
